In [68]:
#Importa modelos
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import confusion_matrix

import numpy as np

In [106]:
from sklearn.datasets import load_digits
digits = load_digits()

from sklearn.datasets import load_iris
iris = load_iris()

#escolhe dataset
dt = iris

In [107]:
def accuracy(matrix):
    trace = matrix.trace()
    total_sum = matrix.sum()
    accuracy = trace/total_sum
    return accuracy

In [108]:
#stratifiedKFold = KFold que conserva a % de cada classe nos folds
from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=5)

scores_logistic = []
scores_svm = []
scores_rf = []

cm_logistic = []
cm_svm = []
cm_rf = []

for train_index, test_index in folds.split(dt.data,dt.target):
    X_train, X_test, y_train, y_test = dt.data[train_index], dt.data[test_index], \
                                       dt.target[train_index], dt.target[test_index]
    
    #Logistic Regression
    logistic_model = LogisticRegression(solver='liblinear',multi_class='ovr')
    logistic_model.fit(X_train, y_train)

    y_pred = logistic_model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)

    scores_logistic.append(logistic_model.score(X_test, y_test)) #.score() retorna accuracy
    cm_logistic.append(cm)
    

    #SVM
    svm_model = SVC(gamma='auto')
    svm_model.fit(X_train, y_train)

    y_pred = svm_model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)

    scores_svm.append(svm_model.score(X_test, y_test))
    cm_svm.append(cm)
    
    #Random Forest
    rf_model = RandomForestClassifier(n_estimators=40)
    rf_model.fit(X_train, y_train)

    y_pred = rf_model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    
    scores_rf.append(rf_model.score(X_test, y_test))
    cm_rf.append(cm)

In [109]:
print("Acuracia do Logistic Regression:", scores_logistic)
print("Média:", np.mean(scores_logistic))
print("Desvio padrão:", np.std(scores_logistic))

Acuracia do Logistic Regression: [1.0, 0.9666666666666667, 0.9333333333333333, 0.9, 1.0]
Média: 0.9600000000000002
Desvio padrão: 0.038873012632301994


In [110]:
print("Acuracia do Logistic Regression:", scores_svm)
print("Média:", np.mean(scores_svm))
print("Desvio padrão:", np.std(scores_svm))

Acuracia do Logistic Regression: [0.9666666666666667, 1.0, 0.9666666666666667, 0.9666666666666667, 1.0]
Média: 0.9800000000000001
Desvio padrão: 0.016329931618554516


In [111]:
print("Acuracia do Logistic Regression:", scores_rf)
print("Média:", np.mean(scores_rf))
print("Desvio padrão:", np.std(scores_rf))

Acuracia do Logistic Regression: [0.9666666666666667, 0.9666666666666667, 0.9333333333333333, 0.9333333333333333, 1.0]
Média: 0.96
Desvio padrão: 0.024944382578492935
